In [1]:
import sys
!{sys.executable} -m pip install --upgrade stepfunctions

     |████████████████████████████████| 58 kB 7.9 MB/s  eta 0:00:01
  Created wheel for stepfunctions: filename=stepfunctions-1.0.0.9-py2.py3-none-any.whl size=70078 sha256=855aaebde550476c70443173670b237893e3555ee98b98abaef911723c6208f6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/33/ed/73/c9e9ecc0de4d615de23fe24ec8d2528bae6ee8340c36cdeb8f
Successfully built stepfunctions
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [21]:
# import libraries
import boto3
import logging
import sagemaker
import stepfunctions

from sagemaker.sklearn.estimator import SKLearn

from stepfunctions.inputs import ExecutionInput
from stepfunctions import steps
from stepfunctions.steps.sagemaker import TrainingStep
from stepfunctions.workflow import Workflow

In [22]:
# sagemaker execution role
sm_role = sagemaker.get_execution_role()

# stepfunctions execution role
stepfunctions_role = 'arn:aws:iam::613630599026:role/datascience-stepfunctions-service-role'

In [23]:
# define boto3 clients
s3_client = boto3.client('s3')

# set logging
stepfunctions.set_stream_logger(level=logging.INFO)

# collect session info
session = sagemaker.Session()
region = boto3.Session().region_name
acount_id = boto3.client('sts').get_caller_identity().get('Account')

# define s3 bucket
resources_bucket = f'datascience-hbo-users'

# sagemaker session
sagemaker_session = sagemaker.Session(default_bucket=resources_bucket)

# name the inference pipeline modek
pipeline_model_name = 'FTPipeLineModel'

# name the Stepfunctions pipeline
pipeline_name = 'FTInferenceRoutine'

In [24]:
# define the data sources
s3_train_data = sagemaker.session.s3_input(s3_data='s3://{}/users/rc/FT_propensity/7_day/snowflake-hbomax-staging/{}'.format(resources_bucket, 'train'), distribution='ShardedByS3Key', content_type= 'text/csv')
s3_test_data = sagemaker.session.s3_input(s3_data='s3://{}/users/rc/FT_propensity/7_day/snowflake-hbomax-staging/{}'.format(resources_bucket, 'test'), distribution='ShardedByS3Key', content_type= 'text/csv')
s3_inf_data = sagemaker.session.s3_input(s3_data='s3://{}/users/rc/FT_propensity/7_day/snowflake-hbomax-staging/{}'.format(resources_bucket, 'new'), distribution='ShardedByS3Key', content_type= 'text/csv')

In [25]:
# Define runtime input.  SageMaker expects unique names for each job, model and endpoint. 
execution_input = ExecutionInput(schema={
    'SKLearnFeaturizerJobName': str
})

In [26]:
# define the SKLearn Preprocessing Estimator
sklearn_featurizer = SKLearn(
    entry_point='featurizer.py',
    role=sm_role,
    train_instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session)

In [27]:
fit_featurizer_step = TrainingStep(
    'Fit Featurizer', 
    estimator = sklearn_featurizer,
    data={
        'train': s3_train_data
    },
    #tags= {'task': 'tune','algorithm':'linlearner'},
    job_name=execution_input['SKLearnFeaturizerJobName']
)

In [28]:
workflow_definition = steps.Chain([fit_featurizer_step])

workflow = Workflow(
    name=pipeline_name,
    definition=workflow_definition,
    role=stepfunctions_role,
    execution_input=execution_input
)

workflow.create()
workflow.update(workflow_definition)

ClientError: An error occurred (AccessDeniedException) when calling the CreateStateMachine operation: User: arn:aws:sts::613630599026:assumed-role/sagemaker-run-notebook-ExecuteNotebookClientRole-1Q9SQ10V8KNNK/SageMaker is not authorized to perform: states:CreateStateMachine on resource: arn:aws:states:us-east-1:613630599026:stateMachine:FTInferenceRoutine